In [ ]:
# Package Imports and CSV Reads

import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
import requests
import pandas as pd
import time
import tqdm

import warnings
warnings.filterwarnings("ignore")

big5_squad_codes = pd.read_csv('big5_MLS_squad_codes.csv')
mls_squad_codes = pd.read_csv('MLS_squad_codes.csv')

from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
#lists to loop through
cats = ['schedule','shooting','keeper','passing','passing_types','gca','defense','possession','misc']
year_range = ['2017-2018','2018-2019','2019-2020','2020-2021','2021-2022','2022-2023']
mls_year_range = ['2018','2019','2020','2021','2022']

In [ ]:
#FB Ref Columns to create dataframes

score_columns = [['Date','Time','Comp','Round','Day','Venue','Result','GF','GA','Opponent','xG','xGA','Possession%','Attendance','Captain','Formation','Referee','Match Report','Notes']]

shooting_columns = [['Date','Time','Comp','Round','Day','Venue','Result','GF','GA','Opponent','Gls','Sh','SoT','SoT%','G/Sh','G/SoT','Avg Shot Distance','Free Kick Shots','Penalties Made','PKatt','xG','Non pen xG','npxG/Sh','G minus xG','npxG-xG','Match Report']]

goalkeeping_columns = [['Date','Time','Comp','Round','Day','Venue','Result','GF','GA','Opponent','SoTA','GA','Saves','Save%','Clean Sheet','Post-ShotxG','PSxGPlusMinus','PKatt','PKAllowed','PKsaved','PKmissed','Passes40ydspluscompleted','Passes40ydsplusAttempted','Passes40ydsplusCmp%','PassesAtt','PassesThrwn','%ofPassesLaunched','AvgPassLen','GoalKicksAtt','GoalKicks%Launched','GoalKicksAvgLen','OppCrossesAtt','OppCrossesStp','OppCrossesStp%','# of Sweeper Actions','AvgDist sweeper Actions','Match Report']]

passing_columns = [['Date','Time','Comp','Round','Day','Venue','Result','GF','GA','Opponent','Passes Completed','Passes Attempted','Pass Completion %','Total Pass Dist','Total Pass Dist Twd Goal','Short Pass Comp','Short Pass Att','Shrt Pass Cmp%','Med Pass Comp','Med Pass Att','Med Pass Cmp %','Lng Pass Cmp','Lng Pass Att','Lng Pass Cmp %','Ast','xAssistedGoals','xAssists','KeyPasses','Attacking Third Passes','Passes Into 18 yd box Comp','Completed Cross','Prog Passes past own 40 yd line','Match Report']]

pass_type_columns = [['Date','Time','Comp','Round','Day','Venue','Result','GF','GA','Opponent','Passes Attempted','Live Passes','Deadball passes','Free Kick Passes','Passes between open defenders','Passes that go 40 plus yds wide','Crosses','Throw Ins','Corner Kicks','Inswinger Corners','Outswinging Corners','Straight Corners','Passes Completed','Offsides','Passes Blocked','Match Report']]

goal_shot_creation_columns = [['Date','Time','Comp','Round','Day','Venue','Result','GF','GA','Opponent','Shot Creating Actions','Live Passes Leading to shot','Deadball Passes Leading to shot','Dribbles leading to shot','Shots leading to another shot','Fouls drawn leading to shot','Def Actions Leading to Shot','Goal Creating Actions','Passes leading to goal','Goals from deadball','Goals from dribble','Goals from other shot','Goals from foul drawn','Goals from defensive action','Match Report']]

defensive_action_columns = [['Date','Time','Comp','Round','Day','Venue','Result','GF','GA','Opponent','Tackles','TacklesWon','Tackles in Def 3rd','Tackles in Mid 3rd','Tackles in Att 3rd','Tackles vs Dribble','Tackles vs Dribble Att','% of Dribblers tackled succesful','# of times dribbled past','Blocks','Shots Blocked','Passes Blocked','Interceptions','Tackles + Int','Clearances','Errors','Match Report']]

possession_columns = [['Date','Time','Comp','Round','Day','Venue','Result','GF','GA','Opponent','Possession','Touches','Touches in Def Pen Area','Touches in Def 3rd','Touches in Mid 3rd','Touches in Att 3rd','Touches in Att Pen Area','Live ball touches','Attempted Take ons','Succesful Take Ons','Take on success rate','Number of times tackled during take on','% of take ons tackled','Carries','Total Carry Distance','Progressive Carrying Distance','Progressive Carries','Carries into final third','Carries into penalty area','Miscontrols','Dispossessions','Passes Received','Progressive Passes Received','Match Report']]

misc_stats_columns = [['Date','Time','Comp','Round','Day','Venue','Result','GF','GA','Opponent','CrdY','CrdR','2CrdY','Fls','Fld','Off','Crs','Int','TklW','PKwon','PKcon','OG','Recov','Won','Lost','Won%','Match Report']]

opp_shooting_columns = [['opp' + x for x in shooting_columns[0]]]

opp_goalkeeping_columns = [['opp' + x for x in goalkeeping_columns[0]]]

opp_passing_columns = [['opp' + x for x in passing_columns[0]]]

opp_pass_type_columns = [['opp' + x for x in pass_type_columns[0]]]

opp_goal_shot_creation_columns = [['opp' + x for x in goal_shot_creation_columns[0]]]

opp_defensive_action_columns = [['opp' + x for x in defensive_action_columns[0]]]

opp_possession_columns = [['opp' + x for x in possession_columns[0]]]

opp_misc_stats_columns = [['opp' + x for x in misc_stats_columns[0]]]

mls_score_columns = [['Date','Time','Comp','Round','Day','Venue','Result','GF','GA','Opponent','xG','xGA','Possession%','Attendance','Captain','Formation','Referee','Match Report','Notes']]

mls_shooting_columns = [['Date','Time','Comp','Round','Day','Venue','Result','GF','GA','Opponent','Gls','Sh','SoT','SoT%','G/Sh','G/SoT','Avg Shot Distance','Free Kick Shots','Penalties Made','PKatt','xG','Non pen xG','npxG/Sh','G minus xG','npxG-xG','Match Report']]

mls_goalkeeping_columns = [['Date','Time','Comp','Round','Day','Venue','Result','GF','GA','Opponent','SoTA','GA','Saves','Save%','Clean Sheet','Post-ShotxG','PSxGPlusMinus','PKatt','PKAllowed','PKsaved','PKmissed','Passes40ydspluscompleted','Passes40ydsplusAttempted','Passes40ydsplusCmp%','PassesAtt','PassesThrwn','%ofPassesLaunched','AvgPassLen','GoalKicksAtt','GoalKicks%Launched','GoalKicksAvgLen','OppCrossesAtt','OppCrossesStp','OppCrossesStp%','# of Sweeper Actions','AvgDist sweeper Actions','Match Report']]

mls_passing_columns = [['Date','Time','Comp','Round','Day','Venue','Result','GF','GA','Opponent','Passes Completed','Passes Attempted','Pass Completion %','Total Pass Dist','Total Pass Dist Twd Goal','Short Pass Comp','Short Pass Att','Shrt Pass Cmp%','Med Pass Comp','Med Pass Att','Med Pass Cmp %','Lng Pass Cmp','Lng Pass Att','Lng Pass Cmp %','Ast','xAssistedGoals','xAssists','KeyPasses','Attacking Third Passes','Passes Into 18 yd box Comp','Completed Cross','Prog Passes past own 40 yd line','Match Report']]

mls_pass_type_columns = [['Date','Time','Comp','Round','Day','Venue','Result','GF','GA','Opponent','Passes Attempted','Live Passes','Deadball passes','Free Kick Passes','Passes between open defenders','Passes that go 40 plus yds wide','Crosses','Throw Ins','Corner Kicks','Inswinger Corners','Outswinging Corners','Straight Corners','Passes Completed','Offsides','Passes Blocked','Match Report']]

mls_goal_shot_creation_columns = [['Date','Time','Comp','Round','Day','Venue','Result','GF','GA','Opponent','Shot Creating Actions','Live Passes Leading to shot','Deadball Passes Leading to shot','Dribbles leading to shot','Shots leading to another shot','Fouls drawn leading to shot','Def Actions Leading to Shot','Goal Creating Actions','Passes leading to goal','Goals from deadball','Goals from dribble','Goals from other shot','Goals from foul drawn','Goals from defensive action','Match Report']]

mls_defensive_action_columns = [['Date','Time','Comp','Round','Day','Venue','Result','GF','GA','Opponent','Tackles','TacklesWon','Tackles in Def 3rd','Tackles in Mid 3rd','Tackles in Att 3rd','Tackles vs Dribble','Tackles vs Dribble Att','% of Dribblers tackled succesful','# of times dribbled past','Blocks','Shots Blocked','Passes Blocked','Interceptions','Tackles + Int','Clearances','Errors','Match Report']]

mls_possession_columns = [['Date','Time','Comp','Round','Day','Venue','Result','GF','GA','Opponent','Possession','Touches','Touches in Def Pen Area','Touches in Def 3rd','Touches in Mid 3rd','Touches in Att 3rd','Touches in Att Pen Area','Live ball touches','Attempted Take ons','Succesful Take Ons','Take on success rate','Number of times tackled during take on','% of take ons tackled','Carries','Total Carry Distance','Progressive Carrying Distance','Progressive Carries','Carries into final third','Carries into penalty area','Miscontrols','Dispossessions','Passes Received','Progressive Passes Received','Match Report']]

mls_misc_stats_columns = [['Date','Time','Comp','Round','Day','Venue','Result','GF','GA','Opponent','CrdY','CrdR','2CrdY','Fls','Fld','Off','Crs','Int','TklW','PKwon','PKcon','OG','Recov','Won','Lost','Won%','Match Report']]

mls_opp_shooting_columns = [['opp' + x for x in shooting_columns[0]]]

mls_opp_goalkeeping_columns = [['opp' + x for x in goalkeeping_columns[0]]]

mls_opp_passing_columns = [['opp' + x for x in passing_columns[0]]]

mls_opp_pass_type_columns = [['opp' + x for x in pass_type_columns[0]]]

mls_opp_goal_shot_creation_columns = [['opp' + x for x in goal_shot_creation_columns[0]]]

mls_opp_defensive_action_columns = [['opp' + x for x in defensive_action_columns[0]]]

mls_opp_possession_columns = [['opp' + x for x in possession_columns[0]]]

mls_opp_misc_stats_columns = [['opp' + x for x in misc_stats_columns[0]]]

In [ ]:
# Score Links (need to change Match-Logs to Scores-and-Fixtures in link builder.)
score_links = []

for year in range(len(year_range)):
    for squad in range(len(big5_squad_codes)):
        link = (f"https://fbref.com/en/squads/{big5_squad_codes['Squad_Code'][squad]}/{year_range[year]}/matchlogs/all_comps/{cats[0]}/{big5_squad_codes['Text'][squad]}")
        score_links.append(link)
        
for year in range(len(mls_year_range)):
    for squad in range(len(mls_squad_codes)):
        link = (f"https://fbref.com/en/squads/{mls_squad_codes['Squad_Code'][squad]}/{mls_year_range[year]}/matchlogs/c22/{cats[0]}/{mls_squad_codes['Text'][squad]}")
        score_links.append(link)
    
def replace_text(links):
    return [s.replace("Match-Logs", "Scores-and-Fixtures") for s in links]

score_links = replace_text(score_links)

# Shooting Links
shooting_links = []

for year in range(len(year_range)):
    for squad in range(len(big5_squad_codes)):
        link = (f"https://fbref.com/en/squads/{big5_squad_codes['Squad_Code'][squad]}/{year_range[year]}/matchlogs/all_comps/{cats[1]}/{big5_squad_codes['Text'][squad]}")
        shooting_links.append(link)
        
for year in range(len(mls_year_range)):
    for squad in range(len(mls_squad_codes)):
        link = (f"https://fbref.com/en/squads/{mls_squad_codes['Squad_Code'][squad]}/{mls_year_range[year]}/matchlogs/c22/{cats[1]}/{mls_squad_codes['Text'][squad]}")
        shooting_links.append(link)
        
# Goalkeeping Links
goalkeeping_links = []

for year in range(len(year_range)):
    for squad in range(len(big5_squad_codes)):
        link = (f"https://fbref.com/en/squads/{big5_squad_codes['Squad_Code'][squad]}/{year_range[year]}/matchlogs/all_comps/{cats[2]}/{big5_squad_codes['Text'][squad]}")
        goalkeeping_links.append(link)
        
for year in range(len(mls_year_range)):
    for squad in range(len(mls_squad_codes)):
        link = (f"https://fbref.com/en/squads/{mls_squad_codes['Squad_Code'][squad]}/{mls_year_range[year]}/matchlogs/c22/{cats[2]}/{mls_squad_codes['Text'][squad]}")
        goalkeeping_links.append(link)
        
# Passing Links
passing_links = []

for year in range(len(year_range)):
    for squad in range(len(big5_squad_codes)):
        link = (f"https://fbref.com/en/squads/{big5_squad_codes['Squad_Code'][squad]}/{year_range[year]}/matchlogs/all_comps/{cats[3]}/{big5_squad_codes['Text'][squad]}")
        passing_links.append(link)
        
for year in range(len(mls_year_range)):
    for squad in range(len(mls_squad_codes)):
        link = (f"https://fbref.com/en/squads/{mls_squad_codes['Squad_Code'][squad]}/{mls_year_range[year]}/matchlogs/c22/{cats[3]}/{mls_squad_codes['Text'][squad]}")
        passing_links.append(link)
        
# Pass Type Links
pass_type_links = []

for year in range(len(year_range)):
    for squad in range(len(big5_squad_codes)):
        link = (f"https://fbref.com/en/squads/{big5_squad_codes['Squad_Code'][squad]}/{year_range[year]}/matchlogs/all_comps/{cats[4]}/{big5_squad_codes['Text'][squad]}")
        pass_type_links.append(link)
        
for year in range(len(mls_year_range)):
    for squad in range(len(mls_squad_codes)):
        link = (f"https://fbref.com/en/squads/{mls_squad_codes['Squad_Code'][squad]}/{mls_year_range[year]}/matchlogs/c22/{cats[4]}/{mls_squad_codes['Text'][squad]}")
        pass_type_links.append(link)
        
# Goal Shot Creation Links
gca_links = []

for year in range(len(year_range)):
    for squad in range(len(big5_squad_codes)):
        link = (f"https://fbref.com/en/squads/{big5_squad_codes['Squad_Code'][squad]}/{year_range[year]}/matchlogs/all_comps/{cats[5]}/{big5_squad_codes['Text'][squad]}")
        gca_links.append(link)
        
for year in range(len(mls_year_range)):
    for squad in range(len(mls_squad_codes)):
        link = (f"https://fbref.com/en/squads/{mls_squad_codes['Squad_Code'][squad]}/{mls_year_range[year]}/matchlogs/c22/{cats[5]}/{mls_squad_codes['Text'][squad]}")
        gca_links.append(link)
        
# Defensive Action Links
defense_links = []

for year in range(len(year_range)):
    for squad in range(len(big5_squad_codes)):
        link = (f"https://fbref.com/en/squads/{big5_squad_codes['Squad_Code'][squad]}/{year_range[year]}/matchlogs/all_comps/{cats[6]}/{big5_squad_codes['Text'][squad]}")
        defense_links.append(link)
        
for year in range(len(mls_year_range)):
    for squad in range(len(mls_squad_codes)):
        link = (f"https://fbref.com/en/squads/{mls_squad_codes['Squad_Code'][squad]}/{mls_year_range[year]}/matchlogs/c22/{cats[6]}/{mls_squad_codes['Text'][squad]}")
        defense_links.append(link)
        
# Possession Links
poss_links = []

for year in range(len(year_range)):
    for squad in range(len(big5_squad_codes)):
        link = (f"https://fbref.com/en/squads/{big5_squad_codes['Squad_Code'][squad]}/{year_range[year]}/matchlogs/all_comps/{cats[7]}/{big5_squad_codes['Text'][squad]}")
        poss_links.append(link)
        
for year in range(len(mls_year_range)):
    for squad in range(len(mls_squad_codes)):
        link = (f"https://fbref.com/en/squads/{mls_squad_codes['Squad_Code'][squad]}/{mls_year_range[year]}/matchlogs/c22/{cats[7]}/{mls_squad_codes['Text'][squad]}")
        poss_links.append(link)
        
# Misc Links
misc_links = []

for year in range(len(year_range)):
    for squad in range(len(big5_squad_codes)):
        link = (f"https://fbref.com/en/squads/{big5_squad_codes['Squad_Code'][squad]}/{year_range[year]}/matchlogs/all_comps/{cats[8]}/{big5_squad_codes['Text'][squad]}")
        misc_links.append(link)
        
for year in range(len(mls_year_range)):
    for squad in range(len(mls_squad_codes)):
        link = (f"https://fbref.com/en/squads/{mls_squad_codes['Squad_Code'][squad]}/{mls_year_range[year]}/matchlogs/c22/{cats[8]}/{mls_squad_codes['Text'][squad]}")
        misc_links.append(link)
        


In [ ]:
# Team & Opp Dataframes
score_df = pd.DataFrame(columns=score_columns, dtype=object)
shooting_df = pd.DataFrame(columns=shooting_columns, dtype=object)
goalkeeping_df = pd.DataFrame(columns=goalkeeping_columns, dtype=object)
passing_df = pd.DataFrame(columns=passing_columns, dtype=object)
pass_type_df = pd.DataFrame(columns=pass_type_columns, dtype=object)
goal_shot_creation_df = pd.DataFrame(columns=goal_shot_creation_columns, dtype=object)
defensive_action_df = pd.DataFrame(columns=defensive_action_columns, dtype=object)
possession_df = pd.DataFrame(columns=possession_columns, dtype=object)
misc_stats_df = pd.DataFrame(columns=misc_stats_columns, dtype=object)

opp_shooting_df = pd.DataFrame(columns=opp_shooting_columns, dtype=object)
opp_goalkeeping_df = pd.DataFrame(columns=opp_goalkeeping_columns, dtype=object)
opp_passing_df = pd.DataFrame(columns=opp_passing_columns, dtype=object)
opp_pass_type_df = pd.DataFrame(columns=opp_pass_type_columns, dtype=object)
opp_goal_shot_creation_df = pd.DataFrame(columns=opp_goal_shot_creation_columns, dtype=object)
opp_defensive_action_df = pd.DataFrame(columns=opp_defensive_action_columns, dtype=object)
opp_possession_df = pd.DataFrame(columns=opp_possession_columns, dtype=object)
opp_misc_stats_df = pd.DataFrame(columns=opp_misc_stats_columns, dtype=object)

test_df = pd.DataFrame(columns=possession_columns, dtype=object)
test_2_df = pd.DataFrame(columns=opp_possession_columns, dtype=object)

In [ ]:
# Score Data
i = 0
# Loop through each link
for link in tqdm.tqdm(score_links):
    time.sleep(3)
     
    try:
        # Use requests to get the HTML content of the page
        res = requests.get(score_links[i])
        soup = BeautifulSoup(res.text, 'html.parser')
        i += 1
        # Find the header for team info - this is different than the rest of the stats categories
        inner_nav = soup.find('div', id='inner_nav')
        header_data = inner_nav.find('a').text


        # Find the table with the specified div id
        table = soup.find('table', {'id': 'matchlogs_for'})

        # Find table body
        table_body = table.find('tbody')

        # Find all the rows in the table
        rows = table_body.find_all('tr')

        # Initialize an empty list to store the data
        data = []

        # Loop through each row and extract the data
        for row in rows:
            cols = row.find_all(['td','th'])
            cols = [col.text.strip() for col in cols]
            data.append(cols)

        # Create a temporary dataframe from the extracted data
        temp_df = pd.DataFrame(data, columns=score_columns)
        temp_df['Team'] = header_data
        score_df = score_df.append(temp_df, ignore_index=True)
        
    except:
        continue

In [ ]:
# Team Shooting Info
i = 0
for link in tqdm.tqdm(shooting_links[-5:]):
    time.sleep(3)
    
    res = requests.get(shooting_links[-5:][i])
    soup = BeautifulSoup(res.text, 'html.parser')
    i += 1
    
    # Find the header for team info
    header_data = soup.find('th', {'data-stat': 'header_for_against'}).text
    
    # Find the table with the specified div id
    table = soup.find('table', {'id': 'matchlogs_for'})
    
    # Find table body
    table_body = table.find('tbody')
    
    # Find all the rows in the table
    rows = table_body.find_all('tr')
    
    # Initialize an empty list to store the data
    data = []
    
    # Loop through each row and extract the data
    for row in rows:
        cols = row.find_all(['td','th'])
        cols = [col.text.strip() for col in cols]
        data.append(cols)

    # Create a temporary dataframe from the extracted data
    temp_df = pd.DataFrame(data, columns=shooting_columns)
    temp_df['Team'] = header_data
    shooting_df = shooting_df.append(temp_df, ignore_index=True)

In [ ]:
# Practice Example scraping possession data for a sample of teams

# Loop through each link
for link in tqdm.tqdm(test_links):
    time.sleep(3)
    
    try:
        # Use requests to get the HTML content of the page
        res = requests.get(link)
        soup = BeautifulSoup(res.text, 'html.parser')

        # Find the header for team info
        header_data = soup.find('th', {'data-stat': 'header_for_against'}).text

        # Find the table with the specified div id
        table = soup.find('table', {'id': 'matchlogs_for'})

        # Find table body
        table_body = table.find('tbody')

        # Find all the rows in the table
        rows = table_body.find_all('tr')

        # Initialize an empty list to store the data
        data = []

        # Loop through each row and extract the data
        for row in rows:
            cols = row.find_all(['td','th'])
            cols = [col.text.strip() for col in cols]
            data.append(cols)

        # Create a temporary dataframe from the extracted data
        temp_df = pd.DataFrame(data, columns=possession_columns)
        temp_df['Team'] = header_data
        test_df = test_df.append(temp_df, ignore_index=True)
    
    except:
        continue
        
        
#Test to get opponent info
# Loop through each link
for link in tqdm.tqdm(test_links):
    time.sleep(3)
    
    try:
        # Use requests to get the HTML content of the page
        res = requests.get(link)
        soup = BeautifulSoup(res.text, 'html.parser')

        # Find the header for team info
        header_data = soup.find('th', {'data-stat': 'header_for_against'}).text

        # Find the table with the specified div id
        table = soup.find('table', {'id': 'matchlogs_against'})

        # Find table body
        table_body = table.find('tbody')

        # Find all the rows in the table
        rows = table_body.find_all('tr')

        # Initialize an empty list to store the data
        data = []

        # Loop through each row and extract the data
        for row in rows:
            cols = row.find_all(['td','th'])
            cols = [col.text.strip() for col in cols]
            data.append(cols)

        # Create a temporary dataframe from the extracted data
        temp_df = pd.DataFrame(data, columns=opp_possession_columns)
        temp_df['Team'] = header_data
        test_2_df = test_2_df.append(temp_df, ignore_index=True)
    
    except:
        continue